# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code 
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [3]:
# Your code here
q = """ 
SELECT *
FROM employees
LIMIT 3;
"""
pd.read_sql(q, conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing


In [4]:
q = """ 
SELECT *
FROM offices
LIMIT 3;
"""
pd.read_sql(q, conn)

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA


In [10]:
q = """ 
SELECT firstName, lastName
FROM employees
JOIN offices
USING(officeCode)
WHERE city = 'Boston'
"""
pd.read_sql(q, conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [11]:
# Your code here
q = """ 
SELECT officeCode, city, count(employeeNumber) AS no_of_employees
FROM offices
LEFT JOIN employees
USING(officeCode)
GROUP BY officeCode;

"""
pd.read_sql(q, conn)

,officeCode,city,no_of_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2
7,27,Boston,0


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [ ]:
"""
Question 1
"""

# Your code here

In [ ]:
"""
Question 2
"""

# Your code here

In [ ]:
"""
Question 3
"""

# Your code here

## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [20]:
# Your code here
q = """ 
SELECT DISTINCT 
firstName || " " || lastName AS employeeName, products.productName
FROM employees AS e
JOIN customers AS c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders USING(customerNumber)
JOIN orderdetails USING(orderNumber)
JOIN products USING(productCode)
ORDER BY employeeName, products.productName;
"""
pd.read_sql(q, conn)

,employeeName,productName
0,Andy Fixter,18th Century Vintage Horse Carriage
1,Andy Fixter,1900s Vintage Bi-Plane
2,Andy Fixter,1900s Vintage Tri-Plane
3,Andy Fixter,1911 Ford Town Car
4,Andy Fixter,1913 Ford Model T Speedster
...,...,...
1363,Steve Patterson,The Mayflower
1364,Steve Patterson,The Queen Mary
1365,Steve Patterson,The Schooner Bluenose
1366,Steve Patterson,The Titanic


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [33]:
# Your code here
q = """ 
SELECT 
    firstName || " " || lastName AS employeeName,
     IFNULL(SUM(orderdetails.quantityOrdered),0) AS total_products_sold
FROM employees AS e
LEFT JOIN customers AS c ON e.employeeNumber = c.salesRepEmployeeNumber
LEFT JOIN orders USING(customerNumber)
LEFT JOIN orderdetails USING(orderNumber)
LEFT JOIN products USING(productCode)
GROUP BY e.employeeNumber --,employeeName
ORDER BY e.lastName, e.firstName;
"""
pd.read_sql(q, conn)

,employeeName,total_products_sold
0,Gerard Bondur,0
1,Loui Bondur,6186
2,Larry Bott,8205
3,Anthony Bow,0
4,Pamela Castillo,9290
5,Jeff Firrelli,0
6,Julie Firrelli,4227
7,Andy Fixter,6246
8,Martin Gerard,4180
9,Gerard Hernandez,14231


In [28]:
q = """ 
SELECT DISTINCT COUNT(*) AS no_of_employees
FROM employees;
"""
pd.read_sql(q, conn)

,no_of_employees
0,23


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [40]:
# Your code here
q = """ 
SELECT 
firstName || " " || lastName AS employeeName, COUNT(orderdetails.productCode) AS sum_orders
FROM employees AS e
JOIN customers AS c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders USING(customerNumber)
JOIN orderdetails USING(orderNumber)
JOIN products USING(productCode)
GROUP BY employeeName
--ORDER BY employeeName
HAVING sum_orders > 200
;
"""
pd.read_sql(q, conn)

,employeeName,sum_orders
0,Barry Jones,220
1,George Vanauf,211
2,Gerard Hernandez,396
3,Larry Bott,236
4,Leslie Jennings,331
5,Pamela Castillo,272


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!